In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('%s/../' % os.getcwd())

import datetime
import pandas as pd
import numpy as np
import getpass

In [2]:
from aluguel_offshore import *

In [3]:
pasta = r'C:\Users\correigu\OneDrive - Banco BTG Pactual S.A\Desktop\aqruivos_offshore'

In [4]:
pasta_2 = r'C:\Users\correigu\OneDrive - Banco BTG Pactual S.A\Desktop\relatorios_finais'

In [5]:
ubs, morgan, pras, bdwm, intc, ticker_diffs = ImportAluguelOffshore(pasta, pasta_2).import_files()

Daily Stock Loan Fee Summary - 20180301.CSV
Daily Stock Loan Fee Summary - 20180302.CSV
Daily Stock Loan Fee Summary - 20180303.csv
Daily Stock Loan Fee Summary - 20180304.csv
Daily Stock Loan Fee Summary - 20180305.csv
Daily Stock Loan Fee Summary - 20180306.csv
Daily Stock Loan Fee Summary - 20180307.csv
Daily Stock Loan Fee Summary - 20180308.csv
Daily Stock Loan Fee Summary - 20180309.csv
Daily Stock Loan Fee Summary - 20180310.csv
Daily Stock Loan Fee Summary - 20180311.csv
Daily Stock Loan Fee Summary - 20180312.csv
Daily Stock Loan Fee Summary - 20180313.csv
Daily Stock Loan Fee Summary - 20180314.csv
Daily Stock Loan Fee Summary - 20180315.csv
Daily Stock Loan Fee Summary - 20180316.csv
Daily Stock Loan Fee Summary - 20180317.csv
Daily Stock Loan Fee Summary - 20180318.csv
Daily Stock Loan Fee Summary - 20180319.csv
Daily Stock Loan Fee Summary - 20180320.csv
Daily Stock Loan Fee Summary - 20180321.csv
Daily Stock Loan Fee Summary - 20180322.csv
Daily Stock Loan Fee Summary - 2

In [ ]:
%%time

for file in os.listdir(pasta):
 
    if 'Ajuste' in file:
        data = file[-14:-4]
        try:
            ubs
        except:
            ubs = pd.read_csv(pasta+'\\'+file)
            ubs['date']=data
        else:
            ubs_1 = pd.read_csv(pasta+'\\'+file)
            ubs_1['date']=data
            ubs = pd.concat([ubs,ubs_1], axis=0)
    
    
    
    elif 'PROP+ASSET' in file:
        try:
            pras
        except:
            pras = pd.read_excel(pasta+'\\'+file)
            pras['date'] = pras.columns[7]
            pras = pras.rename(columns={pras.columns[7]:'shares'})
        else:
            pras_1 = pd.read_excel(pasta+'\\'+file)
            pras_1['date'] = pras_1.columns[7]
            pras_1 = pras_1.rename(columns={pras_1.columns[7]:'shares'})
            pras = pd.concat([pras,pras_1], axis=0)
            
            
    
    elif 'Aluguel INT C' in file:
        try:
            intc
        except:
            intc = pd.read_excel(pasta+'\\'+file)
            intc['date'] = intc.columns[7]
            intc = intc.rename(columns={intc.columns[7]:'shares'})
        else:
            intc_1 = pd.read_excel(pasta+'\\'+file)
            intc_1['date'] = intc_1.columns[7]
            intc_1 = intc_1.rename(columns={intc_1.columns[7]:'shares'})
            intc = pd.concat([intc,intc_1], axis=0)
    
    elif 'Clients BD+WM' in file:
        try:
            bdwm
        except:
            bdwm = pd.read_excel(pasta+'\\'+file)
            bdwm['date'] = bdwm.columns[31]
            bdwm = bdwm.rename(columns={bdwm.columns[31]:'shares'})
        else:
            bdwm_1 = pd.read_excel(pasta+'\\'+file)
            bdwm_1['date'] = bdwm_1.columns[31]
            bdwm_1 = bdwm_1.rename(columns={bdwm_1.columns[31]:'shares'})
            bdwm = pd.concat([bdwm,bdwm_1], axis=0)
            #print (str(bdwm_1.shape)+' ------ '+str(bdwm.shape))
    
    elif ('038CDBD08_IN150DX' in file)|('038CDB895_IN150DX' in file):
        try:
            morgan
        except:
            morgan = pd.read_csv(pasta+'\\'+file, skipfooter=1, engine='python')
        else:
            morgan_1 = pd.read_csv(pasta+'\\'+file, skipfooter=1, engine='python')
            morgan = pd.concat([morgan, morgan_1], axis = 0)
        
    else:
        print (file)


ticker_diffs = pd.read_excel(pasta+'\\'+ 'tickers_diffs.xlsx')

In [ ]:
months_dict = {
    1:'jan',
    2:'fev',
    3:'mar',
    4:'abr',
    5:'mai',
    6:'jun',
    7:'jul',
    8:'ago',
    9:'set',
    10:'out',
    11:'nov',
    12:'dez'
}

def ajuste_ubs(ubs):
    
    ubs['date'] = pd.to_datetime(ubs['date'], format='%Y-%m-%d')
    ubs['date'] = np.where(ubs['date'].dt.weekday==6, ubs['date']-datetime.timedelta(2), ubs['date'])
    ubs['Daily Accrual'] = ((ubs['Daily Accrual'].str.replace('\(', '-')).str.replace('\)', '')).astype(float).copy()
    
    
    ubs['pb'] = 'ubs'
    ubs['month_number'] = ubs['date'].dt.month
    ubs['year'] = ubs['date'].dt.year
    ubs['month']=ubs['month_number'].apply(lambda x: months_dict[x]).copy()
    
    ubs['cusip'] = np.nan
    ubs = ubs[['Reference Account Id' ,'pb', 'date', 'Ticker', 'ISIN', 'cusip', 'Quantity',
               'Daily Accrual', 'Billing CCY', 'month', 'year']].drop_duplicates().reset_index(drop=True)
    ubs.columns = ['conta', 'pb', 'data_ref', 'ticker', 'isin', 'cusip', 'qt', 'accrual', 'currency', 'month', 'year']
    ubs = ubs.drop_duplicates(subset=['conta', 'data_ref', 'ticker', 'qt', 'currency']).reset_index(drop=True)    
    return ubs

def ajuste_calypso(pras, bdwm, intc):
    pras['tipo'] = 'ASSET'
    bdwm['tipo'] = 'WM'
    intc['tipo'] = 'INT C'

    calypso = pd.concat([pras, bdwm, intc], axis=0).reset_index(drop=True)
    
    calypso['date'] = pd.to_datetime(calypso['date'])
    calypso['month_number'] = calypso['date'].dt.month
    calypso['year'] = calypso['date'].dt.year
    calypso['month']=calypso['month_number'].apply(lambda x: months_dict[x])

    calypso = calypso[['Account Name','tipo', 'Agent','date', 'shares', 'Book', 'PRODUCT_CODE.TICKER', 'PRODUCT_CODE.ISIN', 
                       'PRODUCT_CODE.CUSIP', 'Product Currency', 'month', 'year']].reset_index(drop=True)
    calypso.columns = ['conta_pb','conta', 'agent', 'data_ref', 'quantidade', 'book', 'ticker', 'isin', 'cusip', \
                       'currency', 'month', 'year']
    return calypso


def ajuste_morgan(morgan):
    morgan['date'] = pd.to_datetime(morgan['Value Date'], errors='coerce', format = '%m/%d/%Y')

    morgan['pb'] = 'ms'
    morgan['month_number'] = morgan['date'].dt.month
    morgan['year'] = morgan['date'].dt.year
    morgan['month']=morgan['month_number'].apply(lambda x: months_dict[x])

    morgan = morgan[['Account', 'pb', 'date', 'Symbol', 'ISIN', 'Cusip', 'Shares', 'Net Borrow Cost', 'Currency', 'month', 'year']] 

    morgan.columns = ['conta', 'pb', 'data_ref', 'ticker', 'isin', 'cusip', 'qt', 'accrual', 'currency', 'month', 'year']
    
    morgan['data_ref'] = np.where(morgan['data_ref'].dt.weekday==6, morgan['data_ref']-datetime.timedelta(2), morgan['data_ref'])
    morgan = morgan.drop_duplicates().reset_index(drop=True)
    
    return morgan

def get_pb(morgan, ubs):
    ubs = ajuste_ubs(ubs)
    morgan = ajuste_morgan(morgan)
    
    pb = pd.concat([morgan, ubs], axis=0).reset_index(drop=True)
    
    pb.qt = (pb.qt.replace( '[)]','', regex=True )
               .replace( '[(]','-',   regex=True ).astype(float))
    
    
    return pb

# Ajustes base

In [ ]:
pb = get_pb(morgan, ubs)
calypso = ajuste_calypso(pras, bdwm, intc)

In [ ]:
pb.shape[0]

In [ ]:
calypso = calypso[calypso.conta_pb.isin(pb.conta.drop_duplicates())]

In [ ]:
pb = pb[pb.data_ref.dt.weekday<5]
calypso = calypso[calypso.data_ref.dt.weekday<5]

In [ ]:
pb['accrual_adjusted'] = np.where(pb.data_ref.dt.weekday==4, pb['accrual']*3, pb['accrual'])

In [ ]:
# Coloca a soma por dia e por ticker na tabela do calypso original (que está dividida por book)
'''tickers_calypso = calypso.groupby(by=['data_ref', 'ticker']).sum()
tickers_calypso = tickers_calypso.reset_index().drop('year', axis=1)
calypso = pd.merge(calypso, tickers_calypso, on=['data_ref', 'ticker'], how='left')'''

In [ ]:
dict_contas = {
    '038CDB895':'int_c',
    '038CDBD08':'ms_cayman',
    75201835:'ubs_cayman',
    75202213:'ubs_cayman',
    75202215:'ubs_cayman'
}

In [ ]:
pb['acc'] = pb['conta'].apply(lambda x: dict_contas[x])

In [ ]:
tickers_calypso = calypso[['ticker', 'isin', 'cusip']].drop_duplicates().copy()

In [ ]:
tickers_pb = pb[['ticker', 'isin', 'cusip']].drop_duplicates().copy()

In [ ]:
on_isin = pd.merge(tickers_calypso, tickers_pb[tickers_pb['isin'].isnull()==False], on='isin', how='left').drop(['cusip_y'], axis=1)

In [ ]:
on_isin.columns = ['ticker_calypso', 'isin', 'cusip', 'ticker_byisin']

In [ ]:
on_cusip = pd.merge(on_isin,  tickers_pb[tickers_pb['cusip'].isnull()==False], on=['cusip'], how = 'left').drop('isin_y', axis=1)

In [ ]:
on_cusip['ticker_pb'] = np.where(on_cusip['ticker_byisin'].isnull()==False, on_cusip['ticker_byisin'], on_cusip['ticker'])

In [ ]:
mapa_tickers = on_cusip[['ticker_calypso', 'ticker_pb']].dropna().reset_index(drop=True).copy()

In [ ]:
mapa_tickers = mapa_tickers[['ticker_pb', 'ticker_calypso']]

In [ ]:
mapa_dict = mapa_tickers.set_index('ticker_pb')['ticker_calypso'].to_dict()

In [ ]:
def mapeamento(mapa_dict, x):
    try:
        mapa_dict[x]
    except:
        mapa = np.nan
    else:
        mapa = mapa_dict[x]
    
    return mapa

In [ ]:
pb['ticker_ajuste'] = pb['ticker'].apply(lambda x: mapeamento(mapa_dict, x))

In [ ]:
pb[pb['ticker_ajuste'].isnull()==True]['ticker'].drop_duplicates()

In [ ]:
dict_ajustes = {
    'DHT':'DHT US',
    'GOLD':'GOLD US',
    'SIG':'SIG US',
    
}

In [ ]:
calypso[calypso['ticker'].str.contains('SIG')==True]

In [ ]:
pb.columns

In [ ]:
pb.shape

In [ ]:
pb = pb.merge(mapa_tickers, left_on = 'ticker', right_on='ticker_pb', how='left').drop('ticker_pb', axis=1).rename(columns={
    'ticker':'ticker_original_pb',
    'ticker_calypso':'ticker'
})[['conta', 'pb', 'data_ref', 'ticker', 'isin', 'cusip', 'qt', 'accrual',
       'currency', 'month', 'year', 'accrual_adjusted', 'acc', 'ticker_original_pb']]

In [ ]:
pb.shape

# Batimento mensal

In [ ]:
calypso = calypso[((calypso.month=='mar')|(calypso.month=='abr'))&(calypso.year==2018)].reset_index(drop=True)

In [ ]:
pb = pb[((pb.month=='mar')|(pb.month=='abr'))&(pb.year==2018)]

In [ ]:
calypso = pd.merge(calypso, calypso.groupby(by=['conta_pb', 'agent', 'data_ref', 'ticker']).sum().reset_index(), 
         on = ['conta_pb', 'agent', 'data_ref', 'ticker']).drop('year_y', axis=1).rename(columns = {
    'quantidade_x':'quantidade_book',
    'quantidade_y':'quantidade_ticker',
    'year_x':'year',
    'conta':'tipo',
    'conta_pb':'conta',
    'agent':'pb'
})

In [ ]:
calypso['pb'] = calypso['pb'].replace('MORGAN STANLEY & CO', 'ms').replace('UBS SECURITIES LLC', 'ubs')

In [ ]:
pb['id'] = range(1, pb.shape[0] +1)

In [ ]:
batimento = pd.merge(calypso, pb, on=['conta', 'pb', 'data_ref','ticker', 'currency'], how='left')

In [ ]:
batimento

#### nova tentativa!

In [ ]:
batimento = batimento.drop(['isin_y', 'cusip_y', 'month_y', 'year_y'], axis=1).rename(columns={
    'isin_x':'isin',
    'cusip_x':'cusip',
    'month_x':'month',
    'year_x':'year'
})

In [ ]:
batimento.columns

In [ ]:
batimento = batimento[['conta', 'tipo', 'pb', 'data_ref','ticker','book', 'quantidade_book', 'quantidade_ticker', 'qt',
       'accrual', 'accrual_adjusted', 'acc', 'currency','isin', 'cusip', 'month', 'year', 'id']]

In [ ]:
batimento['qt'] = -1*batimento['qt']

In [ ]:
batimento = batimento[batimento['qt'].isnull()==False].reset_index(drop=True)

In [ ]:
batimento

In [ ]:
batidos = batimento[abs(batimento['quantidade_ticker'])==abs(batimento['qt'])].reset_index(drop=True)

In [ ]:
n_batidos_in_tickers_diff = batimento[(batimento['isin'].isin(ticker_diffs['Isin']))\
                                      &(abs(batimento.qt)!=abs(batimento.quantidade_ticker))&\
                                         (batimento.pb=='ms')].reset_index(drop=True)

In [ ]:
n_batidos_in_tickers_diff['quantidade_ticker']=n_batidos_in_tickers_diff.qt

In [ ]:
n_batidos = batimento[(abs(batimento.qt)!=abs(batimento['quantidade_ticker']))&~((batimento['isin'].isin(ticker_diffs['Isin']))\
                                                          &(batimento.pb=='ms'))].reset_index(drop=True)

In [ ]:
n_batidos['check'] = abs(n_batidos.qt/n_batidos.quantidade_ticker)

In [ ]:
n_batidos

In [ ]:
percent = 0.02

In [ ]:
n_batidos_aceitavel = n_batidos[((n_batidos.check<(1+percent))&(n_batidos.check>(1-percent)))].reset_index(drop=True)

In [ ]:
n_batidos_diff_grande = n_batidos[~((n_batidos.check<(1+percent))&(n_batidos.check>(1-percent)))].reset_index(drop=True)

In [ ]:
n_batidos_aceitavel['quantidade_ticker']=n_batidos_aceitavel.qt

In [ ]:
# check de tamanhos
(batidos.shape[0]+n_batidos_in_tickers_diff.shape[0]+n_batidos_aceitavel.shape[0] + \
                             n_batidos_diff_grande.shape[0])/batimento.shape[0]

In [ ]:
(batidos.shape[0]+n_batidos_in_tickers_diff.shape[0]+n_batidos_aceitavel.shape[0])\
                             /batimento.shape[0]

In [ ]:
already_in = batimento['id'].drop_duplicates().sort_values()

In [ ]:
not_in = pb[pb['id'].isin(already_in)==False].reset_index(drop=True)

In [ ]:
not_in.shape[0]/pb.shape[0]

In [ ]:
not_in.columns

In [ ]:
not_in['conta'].drop_duplicates()

In [ ]:
not_in[(not_in.data_ref==pd.to_datetime('2018-03-07'))&(not_in.ticker=='EWZ US')]

In [ ]:
calypso[(calypso.data_ref==pd.to_datetime('2018-03-07'))&(calypso.ticker.str.contains('EWZ'))]

### Ajuste por ticker (março 2018)

In [ ]:
mar_pb = pb[(pb.month=='mar')&(pb.year==2018)].copy()

In [ ]:
mar_calypso = calypso[(calypso.month=='mar')&(calypso.year==2018)].copy()

In [ ]:
check_byticker_pb = mar_pb.groupby(by=['ticker', 'currency']).sum().reset_index().copy()

In [ ]:
check_byticker_calypso = mar_calypso.groupby(by=['ticker', 'currency']).sum().reset_index().copy()

In [ ]:
check_byticker_calypso['ticker'] = check_byticker_calypso['ticker'].str.split().str[0]

In [ ]:
check_tickers = pd.merge(check_byticker_pb, check_byticker_calypso, on = ['ticker', 'currency'], how='left')\
                    .drop(['year_x', 'year_y'], axis=1)

In [ ]:
check_tickers = check_tickers[['ticker', 'currency', 'accrual', 'qt', 'quantidade']]
check_tickers.columns = ['ticker', 'currency', 'accrual', 'qt_pb', 'qt_calypso']

In [ ]:
check_tickers['accrual'] = np.where(check_tickers['accrual']==0, np.nan, check_tickers['accrual'])
check_tickers['qt_pb'] = np.where(check_tickers['qt_pb']==0, np.nan, check_tickers['qt_pb'])
check_tickers['qt_calypso'] = np.where(check_tickers['qt_calypso']==0, np.nan, check_tickers['qt_calypso'])

In [ ]:
check_tickers['diff_total'] = check_tickers.qt_pb-check_tickers.qt_calypso
check_tickers['diff_pct'] = check_tickers.qt_pb/check_tickers.qt_calypso

In [ ]:
batido = check_tickers[(check_tickers.diff_pct==-1)|(check_tickers.diff_pct==1)].reset_index(drop=True)
batido

In [ ]:
on_pb_off_calypso = check_tickers[pd.isnull(check_tickers['qt_calypso'])==True].reset_index(drop=True)
on_pb_off_calypso

In [ ]:
not_batido = check_tickers[(check_tickers.diff_pct<-1)|(check_tickers.diff_pct>1)].reset_index(drop=True)

In [ ]:
not_batido

In [ ]:
not_batido[not_batido['ticker'].isin(ticker_diffs.Stock)==True]

In [ ]:
not_batido[not_batido['ticker'].isin(ticker_diffs.Stock)==False]

In [ ]:
calypso['check_dia'] = 1

In [ ]:
calypso

In [ ]:
pb['check_dia'] = 1

In [ ]:
pb.conta.drop_duplicates()

In [ ]:
pvt_1 = pd.pivot_table(pb, values=['check_dia'],
                     index='data_ref', columns=['acc'])

In [ ]:
pvt_1

In [ ]:
pvt = pd.pivot_table(calypso, values=['check_dia'],
                     index='data_ref', columns=['tipo'])

In [ ]:
pvt

In [ ]:
teste = ajuste_calypso(pras, bdwm, intc)

In [ ]:
morgan = ajuste_morgan(morgan)

In [ ]:
wm = morgan[morgan['conta']=='038CDF9C8']

In [ ]:
wm

In [ ]:
wm['month'] = wm.data_ref.dt.month
wm['year'] = wm.data_ref.dt.year

In [ ]:
wm.groupby(by=['year','month', 'currency']).sum()